In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

23/05/12 15:35:06 WARN Utils: Your hostname, pclab058 resolves to a loopback address: 127.0.1.1; using 147.96.133.21 instead (on interface eno1)
23/05/12 15:35:06 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/12 15:35:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/05/12 15:35:08 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
data_path = "../sample_10e3.json"

In [3]:
df = spark.read.json(data_path)

In [4]:
df

DataFrame[_id: struct<$oid:string>, ageRange: bigint, idplug_base: bigint, idplug_station: bigint, idunplug_base: bigint, idunplug_station: bigint, travel_time: bigint, unplug_hourTime: struct<$date:string>, user_day_code: string, user_type: bigint, zip_code: string]

In [6]:
df.show()

+--------------------+--------+-----------+--------------+-------------+----------------+-----------+--------------------+--------------------+---------+--------+
|                 _id|ageRange|idplug_base|idplug_station|idunplug_base|idunplug_station|travel_time|     unplug_hourTime|       user_day_code|user_type|zip_code|
+--------------------+--------+-----------+--------------+-------------+----------------+-----------+--------------------+--------------------+---------+--------+
|{5cf83b752f3843a0...|       0|         21|            66|            8|              90|        219|{2019-06-01T00:00...|e4d55deb9ac172a8d...|        1|        |
|{5cf83b762f3843a0...|       4|         19|           136|           19|              71|        359|{2019-06-01T00:00...|8a0c4123e924a50a9...|        1|   28039|
|{5cf83b762f3843a0...|       4|         17|            38|            7|              39|        375|{2019-06-01T00:00...|a6a9c1f74a6849600...|        1|   28013|
|{5cf83b762f3843a0...|

In [8]:
df.columns

['_id',
 'ageRange',
 'idplug_base',
 'idplug_station',
 'idunplug_base',
 'idunplug_station',
 'travel_time',
 'unplug_hourTime',
 'user_day_code',
 'user_type',
 'zip_code']

In [7]:
#1 Contar cada tipo
df.groupBy('user_type').count().show()

+---------+-----+
|user_type|count|
+---------+-----+
|        1|  954|
|        3|   36|
|        2|   10|
+---------+-----+



In [9]:
#1 Contar cada tipo
df.groupBy('ageRange').count().show()

+--------+-----+
|ageRange|count|
+--------+-----+
|       0|  491|
|       6|    5|
|       5|  131|
|       1|   19|
|       3|   75|
|       2|   12|
|       4|  267|
+--------+-----+



In [10]:
#1 Contar cada tipo
df.groupBy('idplug_base').count().show()

+-----------+-----+
|idplug_base|count|
+-----------+-----+
|         26|    9|
|         19|   26|
|         22|   38|
|          7|   45|
|         25|    6|
|          6|   33|
|          9|   49|
|         27|    5|
|         17|   37|
|          5|   50|
|          1|   49|
|         10|   31|
|          3|   54|
|         12|   39|
|          8|   44|
|         11|   39|
|          2|   64|
|          4|   49|
|         13|   27|
|         18|   29|
+-----------+-----+
only showing top 20 rows

